<a href="https://colab.research.google.com/github/urielcast/Intro-Cultural-Analytics/blob/master/Whisper_AI_Transcri%C3%A7%C3%A3o_de_Texto_Multil%C3%ADngue_UrielCast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🗣️ **Whisper** - Transcição de áudios para texto (@urielcast)

*Notebook criado por Uriel castellanos: [@urielcast](https://linktr.ee/urielcast).

*Remixado de [Whisper-Blog](https://openai.com/blog/whisper/), [Whisper-GitHub](https://github.com/openai/whisper), [DotCSV](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg)/[1](https://colab.research.google.com/drive/1CvvYPAFemIZdSOt9fhN541esSlZR7Ic6?usp=sharing#scrollTo=opNkn_Lgpat4) e [Singularidade](https://colab.research.google.com/github/viniciusdutra314/CanalSingularidade/blob/main/WhisperAi/Whisper%20Ai%20Singularidade.ipynb).

*Citar como [ABNT]:
CASTELLANOS, Uriel. Whisper AI: Transcrição de Texto Multilíngue. Versão 1 [S. l.], 13 fev. 2023. Disponível em: https://colab.research.google.com/drive/1qBD3-yRId0p2I3uLn26xJ28aRyGOt3OY?usp=sharing. Acesso em: 13 fev. 2023.

📌 Antes de usar:

🛑 *Ativar* **Aceleração por Hardware** com GPU `"Ambiente de Execução" > "Alterar o tipo de ambiente de execução"`

🛑 Se tiver dúvidas, ver: [Link da Microsoft](https://learn.microsoft.com/pt-br/troubleshoot/developer/browsers/core-features/gpu-hardware-acceleration)

📌 Para usar:
1. Execute a instalação dos módulos (aguardar até ficar um check-list verde);
2. Carregar o arquivo de áudio nos arquivos de google.colab (Verificar que o nome esteja sem caracteres especiais ou conter marcas);
3. Nos três pontos do áudio, escolher a opção "copiar caminho", colar dentro das aspas do código: "audio.mp3" (manter as aspas);
4. Execute "criar legenda".
5. Se for transcribir vários arquivos de áudio, utilize o script "Criar várias legendas", para isso deverá ter carregado os arquivos de áudio, logo executar o segundo script que vai gerar um arquivo novo chamado "copiar.txt". Deveremos abrir o arquivo "copiar.txt", copiar todo o conteúdo nele e colar na célula que estará disponível embaixo. Executar essa nova célula que devera ter por nome "Rodar o código gerado na etapa anterior". Aguardar o script gerar os arquivos de texto.
6. Também podemos gerar a tradução do nosso áudio para o inglês, o script "Traducir** áudio original ao inglês. (X ► English)" segue os mesmos parâmetros iniciais, só devemos colocar o caminho do áudio dentro das aspas "/content/audio.mp3".

📌OBS1: O código criará uma nova pasta chamada legendas, nele vão estar disponíveis os diferentes arquivos da transcrição em texto, alguns deles vão ter marcas de tempo, outros não. Para utilizar numa pesquisa, com software de análises de dados, recomendo aquele que contém a extensão ".txt", mas o código gera diferentes extensões como ".json", ".srt", ".tsv", ".txt" e ".vtt".

📌OBS2: O código pode demorar vários minutos, vai depender do tempo do áudio. O que também pode gerar erro na saída do texto, pois a base de dados da whisper contém limites do tempo de uso (aproximadamente 1h).

📌OBS3: A qualidade da transcrição pode variar segundo a opção utilizada: **tiny, base, small, medium, large**. Lembrando que a melhor opção é "large", mas esta opção vai demandar mais tempo para o processamento do áudio em texto. O parâmetro utilizado neste código foi a opção "medium", pois é uma opção que não vai demandar muita memória do seu computador (pc/laptop), recomendo trocar para "large" se seu computador tiver uma melhor placa de vídeo.

📌OBS4: 💾 **Lembre exportar todos os arquivos gerados e fazer/manter um backup, pois o sistema de google.colab redefine os parâmetros quando você reiniciar a página ou fechar ela**.

OBS5: Se tiver vídeos, recomendo transformar eles em áudio com ajuda de udacity (https://www.audacityteam.org/).




In [ ]:
#@title Execute para instalar os módulos necessários.
"""
Para escrever este código, foi necessaria a inspiração/código de vários lugares/sites.
Remixado de
[Whisper-Blog](https://openai.com/blog/whisper/),
[Whisper-GitHub](https://github.com/openai/whisper),
[DotCSV](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg)/
[1](https://colab.research.google.com/drive/1CvvYPAFemIZdSOt9fhN541esSlZR7Ic6?usp=sharing#scrollTo=opNkn_Lgpat4) e
[Singularidade](https://colab.research.google.com/github/viniciusdutra314/CanalSingularidade/blob/main/WhisperAi/Whisper%20Ai%20Singularidade.ipynb).
"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install pytube
!pip install unidecode
!pip install ffmpeg

import os
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

## 👉 Criar uma legenda (no mesmo idioma, X ► X)

In [ ]:
!whisper "audio.mp3" --task transcribe --model large --verbose False --output_dir legendas

## 👉 Criar várias legendas (no mesmo idioma, X ► X)


In [ ]:
audios=[j for j in os.listdir() if j[-4:] in [".mp3",".ogg",".wav"]]
quality=input("Qual modo de qualidade você deseja? (tiny,base,small,medium,large) ")
with open("copiar.txt","w") as text:
    for k in audios: text.write(f"!whisper '{k}' --task transcribe --model {quality} --verbose False --output_dir legendas\n")


## 👉  **Traducir** áudio original ao inglês. (X ► English)




In [ ]:
!whisper "/content/audio.mp3" --task translate --model medium

## 🎙️ **Grabar y transcribir** [Elige la tarea]

In [ ]:
Tarea = "Transcript to Language" #@param ["Transcript to Language", "Translate to English"]
import numpy as np
import whisper
from scipy.io.wavfile import write
from IPython.display import clear_output

task = "translate" if Tarea == "Translate to English" else "transcribe"

audio, sr = get_audio()
write('record.wav', sr, audio)

!whisper "record.wav" --task {task} --model medium --verbose False

clear_output()
if task == "translate":
  print("-- TRADUCCIÓN A INGLÉS --\n")
else:
  print("-- TRANSCRIPCIÓN A ESPAÑOL --\n")

print(open('record.wav.txt').read())

## 🎙️ **Youtube - Transcribir** [Precisa do link]

In [ ]:
# @title Se for um link do YouTube, abra esta célula e atribua o link a variável "url" que apareçá no ínicio do código. Depois disto, basta executar.

url='https://youtu.be/GJMiYHfEqnQ?si=X8ah96oX19GS87hT'

###############################################################################################
## Não editar nada abaixo, apenas a "url" acima.
try:
    yt = YouTube(url)
    stream = yt.streams.filter(only_audio=True, file_extension='mp4').order_by('abr').asc().first()

    if stream:
        filesize_mb = int(stream.filesize / (1024 * 1024)) # Convertendo bytes para MB
        if filesize_mb <= 100:
            print(f'O vídeo {url} tem {filesize_mb} MB.')

            # Construindo o tempo
            seconds = yt.length
            hours, remainder = divmod(seconds, 3600)
            minutes, seconds = divmod(remainder, 60)
            time = f"{hours:02}:{minutes:02}:{seconds:02}"

            print(f'Iniciando download do vídeo {url} com duração de {time}')

            destination_folder = '/content/'
            audio_file = stream.download(output_path=destination_folder)
            mp3_file = os.path.join(destination_folder, 'video_youtube.mp3')
            os.rename(audio_file, mp3_file) # Renomeando o arquivo para 'video_youtube.mp3'

            print(f'Download finalizado e salvo em {mp3_file}')
        else:
            print(f'O vídeo {url} é muito grande para download.')
    else:
        print(f'Não foi possível encontrar uma versão de áudio para o vídeo {url}. Tente baixar o aúdio/video por outro veículo e use a opção de transcrição para MP3 ou MP4.')
except Exception as e:
    print(f"Erro '{e}' ao fazer download do vídeo {url}")


arquivo_audio = mp3_file
arquivo_transcricao = '/content/transcricao_audio_youtube.txt'